In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re
import numpy as np
%matplotlib inline

In [3]:
df = pd.read_excel('/home/dariodg/Desktop/free/Galpao/ComissionApp/docs/comissabril2019.XLS', )

In [4]:
import json
import os

class ConfigManager:
    # config_file = None

    def __init__(self, config_file_json):
        self.config_file = config_file_json
        

    @property
    def workdir(self):
        return os.getcwd()

    @property
    def settings(self):
        with open(self.config_file) as config_file:
            settings = json.load(config_file)
        return settings

In [5]:
class DataEngineer:

    CONFIG_FILE = 'general_config.json'
    SETTINGS = ConfigManager(CONFIG_FILE).settings
    # this line will be replaced by the gui input
    DATASET = pd.read_excel(SETTINGS['dataset'])
    PATTERNS = SETTINGS['classification_patterns']

    # def __init__(self, config_file):
    #     self.settings = ConfigManager(config_file)
    #     self.data_file = pd.read_excel(self.settings['data_file'])

    def _regex_build(self):
        """Builds a regex for get_non_match method. Ex: '(ADES)|(MULT)|(PROD)'."""
        all_patterns = []
        for pattern in self.PATTERNS.values():
            all_patterns.append(f'({pattern})')
        return '|'.join(all_patterns)

    @staticmethod
    def _str_to_float(float_string):
        """
        It takes a float string ("1,23" or "1,234.567.890") and
        converts it to floating point number (1.23 or 1.234567890).
        """
        
        if float_string is not None: 
            float_string = str(float_string)
            if float_string.count(".") == 1 and float_string.count(",") == 0:
                return float(float_string)
            else:
                midle_string = list(float_string)
                while midle_string.count(".") != 0:
                    midle_string.remove(".")
                out_string = str.replace("".join(midle_string), ",", ".")
            return float(out_string)
        return ''

    @property
    def data_info(self):
        return f'''
            "config_file": {self.CONFIG_FILE},
            "config_keys": {tuple(self.SETTINGS.keys())},
            "config_values": {tuple(self.SETTINGS.values())},
            "data_file": {self.SETTINGS["data_file"]}
        '''

    @staticmethod
    def fill_empty(current_value, new_value):
        """ Change empty values, considered as float, to new_value """
        if type(current_value) == float:
            return new_value
        return current_value

    @staticmethod
    def str_normalize(row):
        return row.upper()
    
    @staticmethod
    def strip_n_split(row, separator):
        """Remove white spaces and split new lines, forming a list."""
        return row.strip().split(separator)

    @staticmethod
    def get_match(row, pattern):
        """ 
        Select a value from a list that correspond to a given value, or
        returns empty if no match was found.
        PS: Assign the result to a new variable. Dont override the original
         """
        for value in row:
            if pattern in value:
                return value
        return ''


    def get_non_match(self, row):
        """Select a value from a list that do not correspond a given regex"""
        regex = self._regex_build()
        for value in row:
            if bool(re.search(regex, value)):
                return ''
            return value

    def split_label_n_amount(self, pd_serie):
        """ Split the payment's label and its amount in two columns."""
        df = pd_serie.str.split(pat=r'R\$', expand=True)
        df[0] = df[0].str.strip()
        df[1] = df[1].str.strip()
        df[1] = df[1].apply(self._str_to_float)
        return df[0], df[1]

    def apply_treatment(self):
        """ 
        Main method of the class.
        Called to apply the functions, through pandas, and return a new dataframe.
        """

        # Column 'turma' treatment
        turmas = self.DATASET['turma'].apply(self.fill_empty, args=('sem_turma',))

        # Normalize descritivo column and split each line row string in a list of values.
        descritivo = self.DATASET['descritivo'].apply(self.str_normalize)
        descritivo = descritivo.apply(self.strip_n_split, args=('\n',))

        # Select values of a panda.Series that match a criteria.
        adesao = descritivo.apply(self.get_match, args=(self.PATTERNS['adesao'],))
        producao = descritivo.apply(self.get_match, args=(self.PATTERNS['producao'],))
        multa = descritivo.apply(self.get_match, args=(self.PATTERNS['multa'],))
        desconto = descritivo.apply(self.get_match, args=(self.PATTERNS['desconto'],))

        # Select values of a panda.Series that dont match a criteria.
        parcelas = descritivo.apply(self.get_non_match)

        # Split label and amount
        label_parcelas, amount_parcelas = self.split_label_n_amount(parcelas)
        label_producao, amount_producao = self.split_label_n_amount(producao)
        label_multa, amount_multa = self.split_label_n_amount(multa)
        label_desconto, amount_desconto = self.split_label_n_amount(desconto)
        label_adesao, amount_adesao = self.split_label_n_amount(adesao)


        treated_df = {
            'TURMA': turmas,
            'TITULO': self.DATASET['titulo'],
            'VENCIMENTO': self.DATASET['vencimento'],
            'SACADO': self.DATASET['sacado'],
            'VAL_RECEBIDO':self.DATASET['recebido'],
            'VAL_PARCELAS': amount_parcelas,
            'VAL_PRODUCAO': amount_producao,
            'VAL_MULTA': amount_multa,
            'VAL_DESCONTO': amount_desconto,
            'VAL_ADESAO': amount_adesao,
            'DATA_CREDITO':self.DATASET['credito'],

        }
        
        return treated_df

    def export_excel(self):
        """ Export the treated dataframe to a excel file """
        df = pd.DataFrame(data=self.apply_treatment())
        defaul_save_names = self.SETTINGS['default_save_name']
        if True:
            df.to_excel(defaul_save_names['incomes_table'], index=False)

    def run(self):
        self.apply_treatment()
        self.export_excel()



In [6]:
df_dicto = DataEngineer().apply_treatment()

In [7]:
df = pd.DataFrame(df_dicto)

In [8]:
df.iloc[[12]]

,TURMA,TITULO,VENCIMENTO,SACADO,VAL_RECEBIDO,VAL_PARCELAS,VAL_PRODUCAO,VAL_MULTA,VAL_DESCONTO,VAL_ADESAO,DATA_CREDITO
12,2019 AC - ADOLESC - TURMA 2,73892,05/03/2019,Catarina Schor Macedo,451.67,,,,,16.67,13/3/2019


In [11]:
col_list = list(df)

In [12]:
col_list

['TURMA',
 'TITULO',
 'VENCIMENTO',
 'SACADO',
 'VAL_RECEBIDO',
 'VAL_PARCELAS',
 'VAL_PRODUCAO',
 'VAL_MULTA',
 'VAL_DESCONTO',
 'VAL_ADESAO',
 'DATA_CREDITO']

In [13]:
df['TURMA'].size

219

In [14]:
df2 = pd.read_excel('incomes.xls')

In [16]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228 entries, 0 to 227
Data columns (total 11 columns):
TURMA           219 non-null object
TITULO          219 non-null float64
VENCIMENTO      219 non-null object
SACADO          219 non-null object
VAL_RECEBIDO    219 non-null float64
VAL_PARCELA     219 non-null object
VAL_PRODUCAO    172 non-null float64
VAL_MULTA       18 non-null float64
VAL_DESCONTO    1 non-null float64
VAL_ADESAO      23 non-null float64
DATA_CREDITO    219 non-null object
dtypes: float64(6), object(5)
memory usage: 19.7+ KB


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219 entries, 0 to 218
Data columns (total 11 columns):
TURMA           219 non-null object
TITULO          219 non-null int64
VENCIMENTO      219 non-null object
SACADO          219 non-null object
VAL_RECEBIDO    219 non-null float64
VAL_PARCELAS    219 non-null object
VAL_PRODUCAO    219 non-null object
VAL_MULTA       219 non-null object
VAL_DESCONTO    219 non-null object
VAL_ADESAO      219 non-null object
DATA_CREDITO    219 non-null object
dtypes: float64(1), int64(1), object(9)
memory usage: 18.9+ KB
